## Pandas 익히기 II
- 판다스 사용법을 코드를 쓰며 익혀보도록 하겠습니다.

### `Quiz`
다음의 데이터 테이블을 만들어보도록 하자.
1. 데이터의 컬럼은 
- 결제번호 : 정수 0 ~ 99
- 결제일 2023(임의의 날짜)
- 상품 : 의류, 신발류, 전자제품, 스포츠류, 책류
- 물량 : 정수 (1 이상)
- 가격 : 5000, 5000, 10000, 3000, 3000
- 결제방법 : 신용카드, 현금

2. 총건수는 100건으로 만들어 봅시다.

3. 상품별로 매출액을 산출해보세요.
- hint : groupby 이용하고 agg라고 하는 method를 이용해봅니다.

4. 신용카드, 현금 결제 중에서 어떤 방식의 결제가 횟수가 더 많은지, 결제 규모가 어떤지 구해보세요.

In [1]:
import numpy as np
import pandas as pd
import random

pd.to_datetime("2023/03/21")

Timestamp('2023-03-21 00:00:00')

### 1. 먼제 데이터를 생성해보자.

In [2]:
# seed를 고정하면 랜덤한게 고정된다.
np.random.seed(0)

paymentId = list(range(0,100))
paymentDate = np.random.choice(pd.date_range('20240101', '20241231'), 100)
product = np.random.choice(['의류', '신발류', '전자제품', '스포츠류', '책류'], 100)
quantity = np.random.randint(1,20,100)
# price를 데이터를 만들고난 후 apply를 적용
price = {'의류' : 5000, '신발류' : 5000, '전자제품' : 10000, '스포츠류' : 3000, '책류' : 3000}
payment = np.random.choice(['신용카드', '현금'], 100)

df = pd.DataFrame({
    "paymentId" : paymentId,
    "paymentDate" : paymentDate,
    "product" : product,
    "quantity" : quantity,
    "payment" : payment
})

df

,paymentId,paymentDate,product,quantity,payment
0,0,2024-06-21,책류,6,신용카드
1,1,2024-02-17,전자제품,12,신용카드
2,2,2024-04-27,의류,1,신용카드
3,3,2024-07-11,스포츠류,12,신용카드
4,4,2024-11-19,전자제품,9,신용카드
...,...,...,...,...,...
95,95,2024-04-05,전자제품,12,신용카드
96,96,2024-07-16,스포츠류,5,현금
97,97,2024-04-04,전자제품,1,현금
98,98,2024-09-13,책류,11,신용카드


In [3]:
# price를 추가해주자

df['price'] = df.apply(lambda x : price[x['product']], axis=1)
df

,paymentId,paymentDate,product,quantity,payment,price
0,0,2024-06-21,책류,6,신용카드,3000
1,1,2024-02-17,전자제품,12,신용카드,10000
2,2,2024-04-27,의류,1,신용카드,5000
3,3,2024-07-11,스포츠류,12,신용카드,3000
4,4,2024-11-19,전자제품,9,신용카드,10000
...,...,...,...,...,...,...
95,95,2024-04-05,전자제품,12,신용카드,10000
96,96,2024-07-16,스포츠류,5,현금,3000
97,97,2024-04-04,전자제품,1,현금,10000
98,98,2024-09-13,책류,11,신용카드,3000


위와 같이 데이터를 완성했다.

### 상품별 매출 집계

In [4]:
# 매출액 산정 
# eval을 이용해보자.
# eval은 행내에서 해줘야 하는 연산들을 적어주면 그 결과를 반환해주는 메서드

df.eval("revenue = price * quantity", inplace=True)
df

,paymentId,paymentDate,product,quantity,payment,price,revenue
0,0,2024-06-21,책류,6,신용카드,3000,18000
1,1,2024-02-17,전자제품,12,신용카드,10000,120000
2,2,2024-04-27,의류,1,신용카드,5000,5000
3,3,2024-07-11,스포츠류,12,신용카드,3000,36000
4,4,2024-11-19,전자제품,9,신용카드,10000,90000
...,...,...,...,...,...,...,...
95,95,2024-04-05,전자제품,12,신용카드,10000,120000
96,96,2024-07-16,스포츠류,5,현금,3000,15000
97,97,2024-04-04,전자제품,1,현금,10000,10000
98,98,2024-09-13,책류,11,신용카드,3000,33000


In [5]:
# groupby로 데이터를 집계
# agg를 이용하면 min, max, sum, mean를 한번에 구할 수 있다.
# 여기서는 sum만 이용하겠다.

df.groupby('product').agg({"revenue" : ["sum", "mean"]})

revenue               
             sum           mean
product                        
스포츠류      795000   29444.444444
신발류       550000   42307.692308
의류       1255000   46481.481481
전자제품     2130000  101428.571429
책류        336000   28000.000000

In [66]:
# 신용카드 vs 현금 결제
# 결제횟수는 신용카드가 많고

df.groupby('payment', as_index=False)['paymentId'].count()

,payment,paymentId
0,신용카드,55
1,현금,45


In [67]:
df.head()

,paymentId,paymentDate,product,quantity,payment,price,revenue
0,0,2024-06-21,책류,6,신용카드,3000,18000
1,1,2024-02-17,전자제품,12,신용카드,10000,120000
2,2,2024-04-27,의류,1,신용카드,5000,5000
3,3,2024-07-11,스포츠류,12,신용카드,3000,36000
4,4,2024-11-19,전자제품,9,신용카드,10000,90000


In [8]:
df.groupby("payment", as_index=False).agg({"revenue" : "sum"})

,payment,revenue
0,신용카드,3195000
1,현금,1871000
